In [ ]:
# TEMPORARY: App Deployment Testing Notebook
# This notebook tests ONLY the app deployment part
# Can be deleted after testing is complete

import os, json, base64, requests, subprocess, shutil, platform
from datetime import datetime

# Get user and context info
user = spark.sql("SELECT current_user()").collect()[0][0]
ctx = dbutils.notebook.entry_point.getDbutils().notebook().getContext()
api_url = ctx.apiUrl().get()
token = ctx.apiToken().get()

print("🔧 APP DEPLOYMENT TEST")
print("="*70)
print(f"👤 User: {user}")
print(f"🏢 Workspace: {api_url}")

In [ ]:
# Step 1: Upload App Files
print("\n" + "="*70)
print("📤 STEP 1: UPLOADING APP FILES")
print("="*70)

app_name = "waf-automation-tool"
workspace_path = f"/Users/{user}/waf-app-source"
app_source_dir = os.path.join(os.getcwd(), "streamlit-waf-automation")

# Create directory
mkdir_response = requests.post(
    url=f"{api_url}/api/2.0/workspace/mkdirs",
    headers={"Authorization": f"Bearer {token}", "Content-Type": "application/json"},
    json={"path": workspace_path}
)

# Upload each file
for item in os.listdir(app_source_dir):
    source = os.path.join(app_source_dir, item)
    if os.path.isfile(source):
        dest_path = f"{workspace_path}/{item}"
        
        with open(source, "rb") as f:
            content = f.read()
        
        upload_response = requests.post(
            url=f"{api_url}/api/2.0/workspace/import",
            headers={"Authorization": f"Bearer {token}", "Content-Type": "application/json"},
            json={
                "path": dest_path,
                "content": base64.b64encode(content).decode("utf-8"),
                "format": "AUTO",
                "language": "PYTHON" if item.endswith(".py") else "AUTO",
                "overwrite": True
            }
        )
        
        if upload_response.status_code in [200, 201]:
            print(f"   ✅ Uploaded: {item}")
        else:
            print(f"   ⚠️  Failed to upload {item}: {upload_response.status_code}")

print(f"\n✅ App files uploaded to {workspace_path}")

In [ ]:
# Step 2: Install CLI if needed
print("\n" + "="*70)
print("🔧 STEP 2: CHECKING/INSTALLING DATABRICKS CLI")
print("="*70)

cli_available = shutil.which("databricks") is not None

if not cli_available:
    print(f"   ⚠️  Databricks CLI not found - attempting to install...")
    
    try:
        system = platform.system().lower()
        
        if system == "linux":
            install_cmd = "curl -fsSL https://raw.githubusercontent.com/databricks/setup-cli/main/install.sh | sh"
            result = subprocess.run(
                install_cmd,
                shell=True,
                capture_output=True,
                text=True,
                timeout=60
            )
            
            if result.returncode == 0:
                os.environ["PATH"] = f"{os.path.expanduser('~')}/.databricks/bin:" + os.environ.get("PATH", "")
                cli_available = shutil.which("databricks") is not None
                
                if cli_available:
                    print(f"   ✅ Databricks CLI installed successfully")
                else:
                    print(f"   ⚠️  CLI install may have succeeded but not in PATH")
            else:
                print(f"   ⚠️  CLI installation failed: {result.stderr[:200]}")
        else:
            print(f"   ⚠️  Auto-install not supported for {system}")
    except Exception as e:
        print(f"   ⚠️  Could not install CLI: {e}")
else:
    print(f"   ✅ Databricks CLI found")

if cli_available:
    # Verify CLI version
    try:
        version_result = subprocess.run(
            "databricks --version",
            shell=True,
            capture_output=True,
            text=True,
            timeout=10
        )
        if version_result.returncode == 0:
            print(f"   📦 CLI Version: {version_result.stdout.strip()}")
    except:
        pass

In [ ]:
# Step 3: Deploy App using CLI
print("\n" + "="*70)
print("🚀 STEP 3: DEPLOYING APP")
print("="*70)

app_url = None

if cli_available:
    # Set environment variables for CLI
    env = os.environ.copy()
    env['DATABRICKS_HOST'] = api_url
    env['DATABRICKS_TOKEN'] = token
    env['DATABRICKS_INSECURE_TLS_SKIP_VERIFY'] = 'true'
    
    print(f"   📁 Source path: /Workspace{workspace_path}")
    print(f"   🔐 Using notebook credentials for CLI authentication")
    
    # Deploy app
    deploy_cmd = f'databricks apps deploy {app_name} --source-code-path /Workspace{workspace_path}'
    result = subprocess.run(
        deploy_cmd,
        shell=True,
        capture_output=True,
        text=True,
        env=env,
        timeout=60
    )
    
    if result.returncode == 0:
        print(f"\n✅ App deployed successfully via CLI")
        try:
            if result.stdout.strip():
                output_json = json.loads(result.stdout)
                deployment_id = output_json.get("deployment_id", "")
                if deployment_id:
                    print(f"   🆔 Deployment ID: {deployment_id}")
        except:
            print(f"   📝 Output: {result.stdout[:200]}")
        
        # Get app URL
        get_app_cmd = f'databricks apps get {app_name} --output json'
        app_result = subprocess.run(
            get_app_cmd,
            shell=True,
            capture_output=True,
            text=True,
            env=env,
            timeout=30
        )
        
        if app_result.returncode == 0:
            try:
                app_info = json.loads(app_result.stdout)
                app_url = app_info.get("url", "")
                if app_url:
                    print(f"   🚀 App URL: {app_url}")
            except:
                pass
    else:
        print(f"\n❌ CLI deployment failed (exit code: {result.returncode})")
        if result.stderr:
            print(f"   Error: {result.stderr}")
        if result.stdout:
            print(f"   Output: {result.stdout}")
else:
    print(f"\n⚠️  CLI not available - cannot deploy via CLI")

# Always show manual steps
print(f"\n📝 Manual Deployment Steps:")
print(f"   1. Go to Databricks Apps: {api_url}/apps")
print(f"   2. Click 'Create App' or find existing app '{app_name}'")
print(f"   3. Source code path: /Workspace{workspace_path}")
print(f"   4. Click 'Deploy'")

In [ ]:
# Summary
print("\n" + "="*70)
print("✅ APP DEPLOYMENT TEST COMPLETE")
print("="*70)
print(f"\n📊 Results:")
print(f"   App name: {app_name}")
print(f"   Source path: /Workspace{workspace_path}")
print(f"   CLI available: {cli_available}")
if 'app_url' in locals() and app_url:
    print(f"   🚀 App URL: {app_url}")
else:
    print(f"   📝 App URL: Check manually at {api_url}/apps")
print("\n")